In [1]:
from deepstab.flow import estimate_flow
from deepstab.visualize import animate_sequence
%matplotlib inline

In [2]:
import numpy as np
from matplotlib import  rc
rc('animation', html='html5')

from torchvision import transforms
from deepstab.load import ObjectMaskVideoDataset
from deepstab.pwcnet import PWCNet
import flowiz as fz

In [3]:
frame_dir = '../data/raw/DAVIS/JPEGImages/480p/breakdance'
mask_dir = '../data/raw/DAVIS/Annotations_unsupervised/480p/breakdance'
model_path = '../models/base_pwc/pwc_net.pth.tar'

In [4]:
plain_transforms = transforms.Compose([
    transforms.CenterCrop((512, 512)),
    transforms.Lambda(lambda x: np.array(x)[:, :, ::-1].astype(np.float32) * (1.0 / 255.0)),
    transforms.ToTensor()
])
plain_dataset = ObjectMaskVideoDataset([frame_dir], [mask_dir], transforms=plain_transforms)
sequence = plain_dataset[0]

In [5]:
model = PWCNet('../models/base_pwcnet/network-default.pytorch').cuda().eval()

result_frames = []
result_flows = []
for i in range(len(sequence['frames']) - 1):
    flow = estimate_flow(model, sequence['frames'][i], sequence['frames'][i + 1])
    result_frames.append(sequence['frames'][i].numpy().transpose(1, 2, 0)[:, :, ::-1])
    result_flows.append(fz.convert_from_flow(flow.detach().numpy().transpose(1, 2, 0)))
animate_sequence(result_frames, result_flows)